## 背景介绍

假设你的堂兄已经通过房地产投资赚取了很多钱。由于你对数据科学的兴趣，他愿意与你成为业务合作伙伴。他将提供资金，你将提供预测各种房屋价值的模型。


你问你堂兄，他过去如何预测房地产价值。他说这只是直觉。但是这样直觉的来源是他从过去的房屋中识别出了价格模式，然后他利用这些模式对他正在考虑的新房屋做出预测。


机器学习的工作方式相同。我们将从称为决策树的模型开始。有更高级的模型可以提供更准确的预测。但是决策树很容易理解，它们是数据科学中一些最佳模型的基本构建块。


为简单起见，我们将从最简单的决策树开始。




## 模型概览

为简单起见，我们将从最简单的决策树开始。

![图一](7tsb5b1.png)

它将房屋仅分为两类。所考虑的任何房屋的预测价格为同一类别房屋的历史平均价格。


通过拟合房地产训练数据，以下两个决策树中的哪个更有可能产生？

![图二](prAjgku.png)

左侧的决策树（决策树1）可能更有意义，因为它反映了一个事实，即卧室多的房屋比卧室少的房屋的价格更高。这种模式的最大缺点是，它没有涵盖影响房价的大多数因素，例如浴室数量，地段大小，位置等。

你可以使用具有更多“分支”的树来捕获更多因素。这些被称为“更深”的树。还要考虑每个房屋的总面积的决策树可能如下所示：
![图三](R3ywQsR.png)

你可以通过跟踪决策树来预测任何房屋的价格，并始终选择与房屋特征相对应的路径。房屋的预计价格在树的底部。我们在底部进行预测的点称为__叶__。

## 数据准备

本次实验我们将以美国爱荷华州的房价数据为例。

数据位于文件路径'home-data-for-ml-course/train.csv'中。

我们使用以下命令加载和浏览数据：

In [1]:
import pandas as pd

# 文件读取路径
iowa_file_path = 'home-data-for-ml-course/train.csv'

# 将csv文件读取为home_data变量
home_data = pd.read_csv(iowa_file_path)

用以下命令来查看数据的摘要信息

In [2]:
home_data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


## 建立模型

### 第一步：指定预测目标

选择与销售价格相对应的目标变量。 将其保存到名为y的新变量中。通过打印列的列表以查找所需列的名称。

In [3]:
print(home_data.columns)

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [4]:
y = home_data['SalePrice']


### 第二步：建立X

现在，你将创建一个名为X的具有预测功能的DataFrame。

由于只需要原始数据中的某些列，因此首先将创建一个列表，其中包含要在X中使用的列的名称。

你将只使用列表中的以下列：

> LotArea  
> YearBuilt  
> 1stFlrSF  
> 2ndFlrSF  
> FullBath  
> BedroomAbvGr  
> TotRmsAbvGrd

In [5]:
# 在下面创建特征列表
feature_names = ['LotArea','YearBuilt','1stFlrSF','2ndFlrSF','FullBath','BedroomAbvGr','TotRmsAbvGrd']

# 在数据中选择出对应的特征
X = home_data[feature_names]


### 审视数据

在建立模型之前，快速浏览一下X以确认它是否合理

In [6]:
# 审视数据
# 打印X的统计数据
print(X.describe())

# 打印X头几行数据
print(X.head())

             LotArea    YearBuilt     1stFlrSF     2ndFlrSF     FullBath  \
count    1460.000000  1460.000000  1460.000000  1460.000000  1460.000000   
mean    10516.828082  1971.267808  1162.626712   346.992466     1.565068   
std      9981.264932    30.202904   386.587738   436.528436     0.550916   
min      1300.000000  1872.000000   334.000000     0.000000     0.000000   
25%      7553.500000  1954.000000   882.000000     0.000000     1.000000   
50%      9478.500000  1973.000000  1087.000000     0.000000     2.000000   
75%     11601.500000  2000.000000  1391.250000   728.000000     2.000000   
max    215245.000000  2010.000000  4692.000000  2065.000000     3.000000   

       BedroomAbvGr  TotRmsAbvGrd  
count   1460.000000   1460.000000  
mean       2.866438      6.517808  
std        0.815778      1.625393  
min        0.000000      2.000000  
25%        2.000000      5.000000  
50%        3.000000      6.000000  
75%        3.000000      7.000000  
max        8.000000     14.

### 第三步：划分数据

使用train_test_split函数拆分数据。

为了模型可再现性，在构建模型时为random_state设置一个数值。

In [7]:
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(X,y,random_state=1)

### 第三步：构建并拟合模型

创建一个DecisionTreeRegressor并将其保存为iowa_model。确保已完成从sklearn的相关导入以运行此命令。

然后使用上面保存的X和y中的数据拟合刚刚创建的模型。

In [8]:
from sklearn.tree import DecisionTreeRegressor
#构建模型
#为了模型可再现性，在构建模型时为random_state设置一个数值
iowa_model = DecisionTreeRegressor(random_state=1)

# 拟合模型
iowa_model.fit(X,y)

DecisionTreeRegressor(random_state=1)

### 第四步：用验证数据做出预测

使用X作为数据，使用模型的预测命令进行预测。将结果保存到称为predictions的变量中。

In [9]:
#根据所有验证观察结果进行预测
val_predictions = iowa_model.predict(val_X)

检查来自验证数据的预测和实际值。

In [10]:
print(val_predictions[0:5])

print(val_y[0:5])

[231500. 179500. 122000.  84500. 142000.]
258     231500
267     179500
288     122000
649      84500
1233    142000
Name: SalePrice, dtype: int64


### 第五步：计算验证数据中的平均绝对误差

In [11]:
from sklearn.metrics import mean_absolute_error
val_mae = mean_absolute_error(val_y,val_predictions)

print(val_mae)

31.923287671232877


## 欠拟合和过拟合

本次实验以叶子节点为变量，探究此因素对欠拟合和过拟合的影响。

首先定义一个计算mae的函数，参数包好叶子节点数量。

In [12]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

编写一个循环，尝试从一组可能的值中尝试以下max_leaf_nodes值。

在max_leaf_nodes的每个值上调用get_mae函数。以某种方式存储输出，该方式允许选择max_leaf_nodes的值，该值可为数据提供准确的模型。

In [13]:
candidate_max_leaf_nodes = [5, 25, 50, 100, 250, 500]
# 编写循环以从候选max_leaf_nodes中找到理想的树大小
min_mae = 999999
best_tree_size = 0;
for i in candidate_max_leaf_nodes:
    j=get_mae(i, train_X, val_X, train_y, val_y)
    print('MAE:{}   节点数:{}'.format(j, i))
    if min_mae>j:
        best_tree_size=i
        min_mae=j
# 存储max_leaf_nodes的最佳值（它将是5、25、50、100、250或500）

print('max_leaf_nodes的最佳值{}'.format(best_tree_size))

MAE:35044.51299744237   节点数:5
MAE:29016.41319191076   节点数:25
MAE:27405.930473214907   节点数:50
MAE:27282.50803885739   节点数:100
MAE:27893.822225701646   节点数:250
MAE:29454.18598068598   节点数:500
max_leaf_nodes的最佳值100


## 随机森林

### 引言

决策树使您难以做出决策。一棵有很多叶子的深树将过拟合，因为每个预测都来自其叶子上只有少数房屋的历史数据。但是，只有很少叶子的浅树性能会很差，因为它无法捕获原始数据中尽可能多的差异。

即使是当今最复杂的建模技术，也面临着欠拟合和过度拟合之间的这种张力。但是，许多模型都有巧妙的想法，可以带来更好的性能。

我们将介绍一种与决策树非常相似但在大多数情况表现更好的模型，__随机森林__。

随机森林使用许多树，并且通过平均每个组成树的预测来进行预测。与单个决策树相比，它通常具有更好的预测准确性，并且可以与默认参数一起很好地工作。在日后的学习中我们会学习到更多具有更好性能的模型，但是其中许多模型对于获取正确的参数很敏感。这是随机森林的优点之一。


### 构建模型

我们将通过一下代码简单构建出一个随机森林模型，注意它的MAE。

In [14]:
from sklearn.ensemble import RandomForestRegressor

# 定义模型，设定random_state为1
rf_model = RandomForestRegressor(random_state=1)

# 拟合模型
rf_model.fit(train_X,train_y)
rf_val_predictions = rf_model.predict(val_X)
# 计算随机森林在验证数据上的MAE
rf_val_mae = mean_absolute_error(rf_val_predictions, val_y)

print("Validation MAE for Random Forest Model: {}".format(rf_val_mae))

Validation MAE for Random Forest Model: 21857.15912981083


此随机森林的叶子节点数设定为了最佳值100。决策树在此情况下的MAE为27282.50803885739，而随机森林为21857.15912981083。可见随机森林表现确实比决策树更加优秀。